# Motor Control

**Note: The wiring for this is very important.  Please double-check your connections before proceeding.**

In [3]:
# Import GPIO Libraries
import RPi.GPIO as GPIO
import time
GPIO.cleanup()

In [243]:
GPIO.setmode(GPIO.BCM)

coil_A_1_pin = 12
coil_A_2_pin = 16
coil_B_1_pin = 20
coil_B_2_pin = 21

GPIO.setup(coil_A_1_pin, GPIO.OUT)
GPIO.setup(coil_A_2_pin, GPIO.OUT)
GPIO.setup(coil_B_1_pin, GPIO.OUT)
GPIO.setup(coil_B_2_pin, GPIO.OUT)

#forward_seq = ['1001', '1100', '0110', '0011']
#forward_seq = ['1000', '0100', '0010', '0001']
forward_seq = ['1001', '1100', '1010', '0110', '0101', '0011']
#forward_seq = ['1100', '0100', '0110', '0010', '0011', '0001', '1001', '1000']
reverse_seq = list(forward_seq) # to copy the list
reverse_seq.reverse()


In [280]:
last_step='0000'

def forward(delay, steps):
  for i in range(steps):
    for s in forward_seq:
      set_step(s)
      time.sleep(delay)

def backwards(delay, steps):
  for i in range(steps):
    for s in reverse_seq:
      set_step(s)
      time.sleep(delay)


def set_step(step):
  GPIO.output(coil_A_1_pin, step[0] == '1')
  GPIO.output(coil_A_2_pin, step[1] == '1')
  GPIO.output(coil_B_1_pin, step[2] == '1')
  GPIO.output(coil_B_2_pin, step[3] == '1')
  



In [6]:
set_step('0000')
delay = 2
# Time Delay (ms)
steps = 40
forward(int(delay) / 1000.0, int(steps))

backwards(int(delay) / 1000.0, int(steps))
set_step('0000')

## Servo Control

In [7]:
GPIO.setmode(GPIO.BCM)
GPIO.setup(18, GPIO.OUT)
p = GPIO.PWM(18, 50)

In [8]:
p.start(7.5)

In [347]:
p.ChangeDutyCycle(5)

In [320]:
start_pos = 5.0
hold_pos = 6.45
push_pos = 8.1

## Rotate whole cube:

In [321]:
def rotate_cube(num_turns):
    for i in range(num_turns):
        p.ChangeDutyCycle(start_pos)
        time.sleep(0.25)
        p.ChangeDutyCycle(hold_pos)  # to slow things down a bit
        time.sleep(0.25)
        p.ChangeDutyCycle(push_pos)
        time.sleep(0.6)
        p.ChangeDutyCycle(start_pos)

In [328]:
rotate_cube(3)

## Now try a turn:

In [340]:
def rotate_side(num_turns):
    """ rotates the bottom face a quarter turn for each num_turns
    """
    
    # Grab the cube (or at least hold it still on top)
    p.ChangeDutyCycle(start_pos)
    time.sleep(0.4)
    p.ChangeDutyCycle((hold_pos+start_pos)/2.0)  # move halfway to slow things down
    time.sleep(0.2)
    p.ChangeDutyCycle(hold_pos)
    time.sleep(0.5)
    
    delay = 4  # Motor Time Delay (ms)
    steps = 14 # a quarter turn(ish)
    
    for i in range(num_turns):
        adj = i % 2
        forward(int(delay) / 1000.0, int(steps-adj))
        #print "rotate steps: %s, adj: %s" % (steps, adj)
        time.sleep(.25)
        backwards(delay/1000.0, 1)
        set_step('1001')
    
    p.ChangeDutyCycle(start_pos)
    time.sleep(0.3)

In [356]:
def straighten_cube(wiggle=2):
    # Grab the cube (or at least hold it still on top)
    p.ChangeDutyCycle(start_pos)
    time.sleep(0.4)
    p.ChangeDutyCycle((hold_pos+start_pos)/2.0)  # move halfway to slow things down
    time.sleep(0.2)
    p.ChangeDutyCycle(hold_pos)
    time.sleep(0.5)
    
    forward(int(delay) / 1000.0, 2)
    #print "rotate steps: %s, adj: %s" % (steps, adj)
    time.sleep(.25)
    backwards(delay/1000.0, 2)
    set_step('1001')
    
    time.sleep(0.2)
    p.ChangeDutyCycle(start_pos)
    time.sleep(0.4)

In [357]:
rotate_side(1)
straighten_cube(4)

rotate steps: 14, adj: 0


In [375]:
def moves_seq(move_str):
    moves = move_str.split(' ')
    for move in moves:
        if move=='c':
            rotate_cube(1)
        elif move=='s':
            rotate_side(1)
            straighten_cube()
            

In [376]:
moves_seq('c s c s s c s')

rotate steps: 14, adj: 0
rotate steps: 14, adj: 0
rotate steps: 14, adj: 0
rotate steps: 14, adj: 0


In [184]:
set_step('0000')

In [162]:
p.stop()
GPIO.cleanup()